In [6]:
import os
print(os.getcwd())


d:\course\thesis\Project2\QCD


In [1]:
from algorthims import *

d:\programing\Anaconda\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
d:\programing\Anaconda\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
d:\programing\Anaconda\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [3]:
def QASM_generator(circuitname, max_qubit):
    # 创建目录
    directory = "Circuits"
    if not os.path.exists(directory):
        os.makedirs(directory)

    # 根据circuitname选择对应的函数
    circuit_func = None
    if circuitname == "grover":
        circuit_func = grover
    elif circuitname == "qft":
        circuit_func = qft
    elif circuitname == "qpe":
        circuit_func = qpe
    else:
        print("Unsupported circuit name.")
        return

    for n in range(2, max_qubit + 1):
      
        circuit = circuit_func(n)
       
        qasm_str = circuit.qasm()

        filename = os.path.join(directory, f"{circuitname}_{n}.qasm")
        with open(filename, "w") as file:
            file.write(qasm_str)
        print(f"Saved {filename}")

In [10]:
QASM_generator('qft',20)
QASM_generator('qpe',20)
QASM_generator('grover',20)

Saved Circuits\qft_2.qasm
Saved Circuits\qft_3.qasm
Saved Circuits\qft_4.qasm
Saved Circuits\qft_5.qasm
Saved Circuits\qft_6.qasm
Saved Circuits\qft_7.qasm
Saved Circuits\qft_8.qasm
Saved Circuits\qft_9.qasm
Saved Circuits\qft_10.qasm
Saved Circuits\qft_11.qasm
Saved Circuits\qft_12.qasm
Saved Circuits\qpe_2.qasm
Saved Circuits\qpe_3.qasm
Saved Circuits\qpe_4.qasm
Saved Circuits\qpe_5.qasm
Saved Circuits\qpe_6.qasm
Saved Circuits\qpe_7.qasm
Saved Circuits\qpe_8.qasm
Saved Circuits\qpe_9.qasm
Saved Circuits\qpe_10.qasm
Saved Circuits\qpe_11.qasm
Saved Circuits\qpe_12.qasm
Saved Circuits\grover_2.qasm
Saved Circuits\grover_3.qasm
Saved Circuits\grover_4.qasm
Saved Circuits\grover_5.qasm
Saved Circuits\grover_6.qasm
Saved Circuits\grover_7.qasm
Saved Circuits\grover_8.qasm
Saved Circuits\grover_9.qasm
Saved Circuits\grover_10.qasm
Saved Circuits\grover_11.qasm
Saved Circuits\grover_12.qasm
